In [133]:
import glob
import os
from utils import  test_pred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import read_pred_file, test_pred, box_iou, NMS

In [139]:
imgs = glob.glob("Data/Test_DS/images/*")
imgs.sort()
pred_folders = glob.glob("Data/preds/Test*/labels")
pred_folders.sort()
len(pred_folders), len(imgs)

(5, 421)

In [131]:
conf_th = 1

nb_fp = {}
nb_tp = {}
nb_fn = {}

cc = {}

for conf_th in range(35):

    conf_th/=10

    nb_fp[f"combo_{conf_th}"] = 0
    nb_tp[f"combo_{conf_th}"] = 0
    nb_fn[f"combo_{conf_th}"] = 0
    cc[f"combo_{conf_th}"] = []
    
    for file in imgs:
        label_file = file.replace("images","labels").replace('.jpg','.txt')
        labels = read_pred_file(label_file)
        preds = np.zeros((0,5))
        for pred_folder in pred_folders[1:]:
            pred_file = os.path.join(pred_folder, os.path.basename(label_file))
            p = read_pred_file(pred_file)
            preds = np.concatenate([preds, p])
        if len(preds)>1:
            best_boxes = NMS(preds)
            ious = box_iou(best_boxes[:,:4], preds[:,:4])
            best_boxes_scores = np.array([sum(preds[iou>0,4]) for iou in ious.T])
            combine_preds = best_boxes[best_boxes_scores>conf_th,:]
        else:
            combine_preds=preds[preds[:,4]>conf_th,:]

        if len(combine_preds):
            if max(combine_preds[:,-1])>4.5:
                print(combine_preds)
        
        fp, tp, fn = test_pred(labels, combine_preds, conf_th*0)
        nb_fp[f"combo_{conf_th}"] += fp
        nb_tp[f"combo_{conf_th}"] += tp
        nb_fn[f"combo_{conf_th}"] += fn
        if len(combine_preds):
            cc[f"combo_{conf_th}"].append(np.max(combine_preds[:,-1]))

    

    


In [132]:
df = pd.DataFrame()
models = []
tp_s = []
fp_s = []
fn_s = []
precision =  []
recall = []
for k,v in nb_fp.items():
    models.append(k.split('_')[-1])
    fp_s.append(nb_fp[k])
    tp_s.append(nb_tp[k])
    fn_s.append(nb_fn[k])
    precision.append((nb_tp[k])/(nb_tp[k] + nb_fp[k] + 1e-12))
    recall.append((nb_tp[k])/(nb_tp[k] + nb_fn[k] + 1e-12))


df["model"]=models
df["tp"]=tp_s
df["fn"]=fn_s
df["fp"]=fp_s
df["precesion"] = precision
df["recall"] = recall
precision = np.array(precision)
recall = np.array(recall)
f1 = 2*precision*recall / (precision + recall)
df["f1"] = f1
#df.sort_values('f1')
df

,model,tp,fn,fp,precesion,recall,f1
0,0.0,190,27,156,0.549133,0.875576,0.674956
1,0.1,183,34,120,0.603960,0.843318,0.703846
2,0.2,176,41,80,0.687500,0.811060,0.744186
3,0.3,168,48,59,0.740088,0.777778,0.758465
4,0.4,165,51,51,0.763889,0.763889,0.763889
5,0.5,161,55,39,0.805000,0.745370,0.774038
6,0.6,159,57,34,0.823834,0.736111,0.777506
7,0.7,155,61,30,0.837838,0.717593,0.773067
8,0.8,152,64,24,0.863636,0.703704,0.775510
9,0.9,144,72,22,0.867470,0.666667,0.753927
